# Traitement du Langage Naturel (NLP)

![deep](asset/deep.png)

# Objectif

Le but du NLP est d'exploiter et d'analyser le langage naturel à l'aide de l'intelligence artificielle.
Voici une illustration conceptuelle de cette idée :

![deep](asset/nlp.png)

# Entrées

Dans le domaine de la vision par ordinateur, nous utilisons des images comme entrée.

Dans d'autres domaines, nous pouvons utiliser des séries temporelles (tableaux de données numériques) comme entrée.

**Mais dans le domaine du Traitement du Langage Naturel (NLP), comment transformer du texte en une représentation numérique ?**

In [ ]:
!pip install -q -U numpy pandas scikit-learn plotly spacy
!python -m spacy download en_core_web_sm
!python -m spacy download fr_core_news_sm

In [1]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

corpus = ["frites j'aime les frites", "LISP c'est trop bien !", "j'aime les jeux dragon's Lair"]

X = vectorizer.fit_transform(corpus)
print("Matrix", X.toarray())
print("Vocabulary", vectorizer.vocabulary_)

Matrix [[1 0 0 0 2 0 0 1 0 0]
 [0 1 0 1 0 0 0 0 1 1]
 [1 0 1 0 0 1 1 1 0 0]]
Vocabulary {'frites': 4, 'aime': 0, 'les': 7, 'lisp': 8, 'est': 3, 'trop': 9, 'bien': 1, 'jeux': 5, 'dragon': 2, 'lair': 6}


En format tableau

In [ ]:
import pandas as pd

df = pd.DataFrame(data=X.toarray(), columns=vectorizer.get_feature_names_out())
print(df)

Top n mots

In [ ]:
import plotly.graph_objects as go


def get_top_n_words(corpus, n=None):
    vec = CountVectorizer().fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq = sorted(words_freq, key=lambda x: x[1], reverse=True)
    return words_freq[:n]


common_words = get_top_n_words(corpus, 30)
df = pd.DataFrame(common_words, columns=['unigram', 'count'])

fig = go.Figure([go.Bar(x=df['unigram'], y=df['count'])])
fig.update_layout(title=go.layout.Title(text="Top 30 unigrams"))
fig.show()

## Text similarity

In [ ]:
from sklearn.metrics import jaccard_score
from sklearn.feature_extraction.text import CountVectorizer

# Corpus de textes que nous allons utiliser pour trouver des similarités
corpus = ["j'aime les frites",
          "LISP c'est trop bien !",
          "j'aime les jeux dragon's Lair",
          "j'adore les frites",
          "envoyer un mail",
          "mangé saussise"]

# Vectorisation du corpus de texte :
# Cette étape transforme chaque phrase en un vecteur de caractéristiques où chaque dimension
# correspond à un mot unique (caractéristique) présent dans l'ensemble du corpus.
vectorizer = CountVectorizer()

# Apprend le vocabulaire dans le corpus et transforme chaque phrase du corpus en vecteur
X = vectorizer.fit_transform(corpus)

# Conversion de la matrice sparse générée par scikit-learn en tableau numpy pour une manipulation plus simple
arr = X.toarray()


def simlarity_search(arr, input_text):
    """
    Fonction qui recherche les phrases les plus similaires dans le corpus en utilisant
    le score de similarité de Jaccard.

    Arguments :
    - arr : tableau numpy des vecteurs représentant le corpus
    - input_text : texte d'entrée pour lequel on veut calculer la similarité
    """
    # Vectorisation de la phrase d'entrée :
    # Elle est transformée en vecteur pour avoir les mêmes dimensions que ceux du corpus.
    input_text = vectorizer.transform([input_text]).toarray()[0]

    # Calcul du score de similarité de Jaccard pour chaque phrase du corpus
    # Le score de Jaccard compare les similarités entre deux ensembles
    # (dans ce cas, les mots des phrases transformés en vecteur).
    scores = []
    for idx in range(arr.shape[0]):  # Pour chaque vecteur dans le corpus
        # Calcul du score de similarité de Jaccard entre la phrase d'entrée et une phrase du corpus
        score = jaccard_score(input_text, arr[idx])

        # Ajoute le score et la phrase correspondante dans la liste pour un traitement ultérieur
        scores.append([score, corpus[idx]])

    # Trie des scores par ordre décroissant pour afficher les phrases les plus similaires en premier
    scores = sorted(scores, key=lambda x: x[0], reverse=True)

    # Affichage des scores et des phrases correspondantes
    for score, sentence in scores:
        print(f"{round(score, 2)}: {sentence}")


q1 = "j'aime manger des frites"
print(f"Query: {q1}")
# Exécution de la fonction avec un premier texte d'entrée
simlarity_search(arr, q1)
print("----------------------")
q2 = "envoyé des email"
print(f"Query: {q2}")
# Exécution de la fonction avec un second texte d'entrée
simlarity_search(arr, q2)

# Nettoyage des Données (Data Cleaning)

Les données textuelles présentent des défis car un même mot peut apparaître sous différentes formes :
- **Majuscule/minuscule** : `Chat, chat`
- **Conjugaisons** : `aider`, `aidant`, `aidé`, `utile`
- **Autres complexités** : la ponctuation et les mots vides (*stop words*) tels que **le**, **que**, **et autres**.

**Objectif** : Réduire la variété des mots distincts tout en préservant leur sens essentiel afin de simplifier l'analyse.

Nous utiliserons la bibliothèque [spaCy](https://spacy.io) pour nettoyer les données textuelles.

In [ ]:
corpus = ["j'aime, les frites",
          "comment installer un site internet ?",
          "LISP c'est trop bien !",
          "j'aime les jeux dragon's Lair",
          "j'adore les frites",
          "Envoyer un mail",
          "mangé: saussise"]

In [ ]:
import spacy

nlp = spacy.load("fr_core_news_sm")

## Tokeniseur
L'objectif est de diviser une phrase en tokens (unités lexicales).
Par exemple :
`I love to play dragon lair` --> `I`, `love`, `to`, `play`, `dragon`, `lair`

In [ ]:
docs = [nlp(sentence) for sentence in corpus]
for token in docs[0]:
    print(token)

## Mots Vides (StopWords)
Les mots vides (stop words) sont des mots courants qui apportent peu d'information dans un document texte. Des mots comme `le`, `est`, `un(e)` ont une valeur limitée et ajoutent du bruit aux données textuelles.

In [ ]:
for sentence in docs:
    clean_sentence = []
    for token in sentence:
        if not token.is_stop:
            clean_sentence.append(str(token))
    print(' '.join(clean_sentence))

## Ponctuation

Supprimer la ponctuation peut être utile dans certains cas. Cependant, pour d'autres techniques d'encodage, comme celles basées sur l'apprentissage profond, ce n'est généralement pas la meilleure solution.

In [ ]:
for sentence in docs:
    clean_sentence = []
    for token in sentence:
        if not token.is_stop and not token.is_punct:
            clean_sentence.append(str(token))
    print(' '.join(clean_sentence))

## Lemmatization

The goal is to converting a word to its root form  `help`, `helping`, `helped`, `helpful`.

In [ ]:
for sentence in docs:
    clean_sentence = []
    for token in sentence:

        if token.is_stop or token.is_punct:
            continue

        if token.lemma_ != "-PRON-":
            lem_word = token.lemma_.lower()
        else:
            lem_word = token.lower_

        clean_sentence.append(str(lem_word))

    print(' '.join(clean_sentence))

# Bag of words with data cleaning



In [ ]:
clean_sentences = []


def clean_sentence(sentence, nlp):
    clean_sentence = []
    for token in nlp(sentence):

        if token.is_stop or token.is_punct:
            continue

        if token.lemma_ != "-PRON-":
            lem_word = token.lemma_.lower()
        else:
            lem_word = token.lower_

        clean_sentence.append(str(lem_word))

    return ' '.join(clean_sentence)


corpus = ["j'aime les frites",
          "LISP c'est trop bien !",
          "j'aime les jeux dragon's Lair",
          "j'adore les frites",
          "envoyer un mail",
          "mangé saussise"]
docs = [clean_sentence(sentence, nlp) for sentence in corpus]
docs

In [ ]:
# Vectorise the corpus
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(docs)
arr = X.toarray()

print(f"Query: {q1}")
# Exécution de la fonction avec un premier texte d'entrée
simlarity_search(arr, clean_sentence(q1, nlp))
print("----------------------")
print(f"Query: {q2}")
# Exécution de la fonction avec un second texte d'entrée
simlarity_search(arr, clean_sentence(q2, nlp))

# TF-IDF (Fréquence Terme-Inverse de la Fréquence Document)

TF-IDF **Vectorizer** et Count **Vectorizer** sont des méthodes utilisées en traitement automatique du langage naturel (NLP) pour transformer du texte en représentations numériques (vectorisation). Cependant, ces deux approches diffèrent fondamentalement :

- **CountVectorizer** : Compte simplement le nombre de fois qu'un mot apparaît dans un document (approche sac de mots, ou *bag-of-words*).
- **TF-IDF Vectorizer** : Prend en compte la fréquence du mot dans un document mais ajuste cette fréquence en fonction de son importance dans l'ensemble du corpus. Les mots fréquents dans tous les documents (tels que "le", "est", "et") sont pénalisés car ils portent généralement moins d'information.

Il n'y a pas une méthode meilleure que l'autre. Le choix dépend du contexte et des besoins de l'application. Tester les deux est une bonne pratique.


## Comment cela fonctionne ?

**Tf** signifie *fréquence du terme* (Term Frequency), tandis que **TF-IDF** signifie *fréquence terme* multipliée par *inverse de la fréquence document* :

![Bag of Words - Image](./asset/tf_idf_formul.png)


## Terminologie clé

- **t** — terme (mot)
- **d** — document (ensemble de mots)
- **N** — nombre total de documents dans le corpus
- **Corpus** — ensemble global de documents


## Fréquence du terme (TF - Term Frequency)

La fréquence du terme est calculée comme suit :
`tf(t, d) = (nombre d'occurrences de t dans d) / (nombre total de mots dans d)`


## Fréquence Document (DF - Document Frequency)

Le nombre de documents où le terme **t** apparaît est appelé fréquence document.
`df(t) = nombre de documents contenant t`


## Inverse de la Fréquence Document (IDF)

Certains termes comme "le", "est", "de" apparaissent fréquemment mais sont peu informatifs. Pour réduire leur impact, l'inverse de la fréquence document (IDF) est utilisé :

`idf(t) = log(N / (df + 1))`


## Formule finale

Ainsi, la pondération TF-IDF se calcule comme suit :

`tf-idf(t, d) = tf(t, d) * log(N / df)`


## Exemple d'application

- **Phrase A** : "The car is driven on the road."
- **Phrase B** : "The truck is driven on the highway."

(Se référer à l'image associée pour l'exemple pratique.)
![TF-IDF - Exemple](./asset/tf_idf_example.png)


## Similarité vectorielle

Une fois que chaque document est représenté sous forme de vecteurs dans un espace multidimensionnel, on peut mesurer leur similarité à l'aide de la similitude cosinus :

![Similitude Cosine - Formule](./asset/vector_sim.png)

Pour deux vecteurs, **a** et **b**, le cosinus de l'angle (θ) est utilisé pour évaluer leur proximité dans un espace vectoriel :

- **−1** : Les vecteurs sont opposés, ce qui indique aucune similarité.
   *Exemple* : "nord" et "sud".
- **0** : Les vecteurs sont indépendants ou orthogonaux.
   *Exemple* : "chien" et "lune" n'ont aucune relation contextuelle.
- **+1** : Les vecteurs sont similaires ou identiques.
   *Exemple* : "heureux" et "joyeux" expriment des émotions positives proches.

![Cosinus - Formule](./asset/formule_cosine.png)

**||A||** représente la norme Euclidienne du vecteur, qui se calcule comme suit :
![Norme Euclidienne](./asset/euclide.png)


## Cas d'utilisation

TF-IDF peut être appliqué à divers problèmes en NLP :
- Recherche d'information (systèmes de moteurs de recherche).
- Classification de documents.
- Résumé automatique de texte.

In [2]:
print(docs)
print(len(docs))

NameError: name 'docs' is not defined

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

vectorizer = TfidfVectorizer(max_features=200)
vectors = vectorizer.fit_transform(docs)
# liste des mots 't'
vectorizer.vocabulary_

In [ ]:
# 6 phrase
vectors.toarray()

In [ ]:
def tfidf_simlarity_search(vectorizer, dataset_matrix, dataset, input_text):
    # Vectorise l'entrée textuelle (input_text)
    # La méthode `transform` du vectoriseur TF-IDF transforme le texte d'entrée en vecteur sparse (creux),
    # qui contient les poids TF-IDF pour chaque mot selon le vocabulaire du dataset.
    query_vec = vectorizer.transform([input_text])

    # Calcule la similarité cosinus entre la matrice du dataset existant (dataset_matrix)
    # et le vecteur de la requête (query_vec).
    # La similarité cosinus mesure à quel point deux vecteurs sont proches dans l'espace vectoriel,
    # avec une valeur de 1 indiquant une correspondance parfaite.
    results = cosine_similarity(dataset_matrix, query_vec).reshape((-1,))

    print(f"Query: {input_text}")

    # Trie les indices des résultats de la similarité dans l'ordre décroissant
    # afin d'obtenir les 10 documents les plus pertinents.
    # argsort() trie les indices par la valeur associée, et [::-1] inverse cet ordre.
    for i in results.argsort()[-10:][::-1]:
        # Affiche les indices des documents du dataset les plus similaires (ajuste l'indice pour correspondre à une base 1)
        # ainsi que leur contenu.
        print(f"{i + 1} - {dataset[i]}")


query = clean_sentence("manger", nlp)
# Recherche les documents les plus similaires à cette requête en utilisant la fonction TF-IDF.
tfidf_simlarity_search(vectorizer, vectors, docs, query)

# Séparation des résultats pour faciliter la lecture.
print("----------------------")

# Prépare une nouvelle requête utilisateur, cette fois pour "envoyé des email".
query = clean_sentence("envoyé des email", nlp)
# Recherche des documents similaires avec la nouvelle entrée textuelle.
tfidf_simlarity_search(vectorizer, vectors, docs, query)

# Classificateur
## k-NN : Un Classificateur Simple

Le classificateur k-Nearest Neighbor (k-NN) est l'un des algorithmes de classification les plus simples en apprentissage automatique et pour la classification d'images. En réalité, cet algorithme ne "découvre" ou "apprend" rien.

Il repose simplement sur la distance entre les vecteurs de caractéristiques (ici, les intensités brutes des pixels RGB des images).

Pour plus de détails, consultez ce [guide utilisateur](https://scikit-learn.org/stable/modules/neighbors.html#nearest-neighbors-classification).

![Représentation d'un sac](./asset/knn.png)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.neighbors import KNeighborsClassifier

# Jeu de données multilabel
x = ["j'aime les pomme vert", "les orange sont pas top", "une grosse poire", "la belle poire orange"]
y = [['apple', 'green'], ['orange'], ['pear', 'green'], ['pear', 'orange']]

# Jeu de données de test
x_test = ["pomme vert bio", "je suis orange", "la belle orange poire"]
y_test = [['apple', 'green'], ['orange'], ['pear', 'orange']]

# Encodage des classes en multilabel
# Ici, on utilise MultiLabelBinarizer pour transformer les listes de labels en représentations binaires
# Chaque classe est représentée comme 0 (absence) ou 1 (présence), formant un tableau binaire par exemples.
encoder = MultiLabelBinarizer()
y_encoded = encoder.fit_transform(y)  # Encodage des labels du jeu d'entraînement
y_test_encoded = encoder.transform(y_test)  # Encodage des labels du jeu de test (sur la base des classes apprises)

print(f"""Encoder
{encoder.classes_}
----------
Train Y encode
{y_encoded}
----------
Test Y encode
{y_test_encoded}
""")

# Pipeline avec une vectorisation TF-IDF et un classifieur k-NN
model_pipeline = Pipeline([
    # Étape 1 : Transformation des textes en vecteurs numériques avec TF-IDF
    # - TF-IDF (Term Frequency-Inverse Document Frequency) aide à quantifier les mots tout en diminuant le poids des mots plus communs.
    # - `max_features=200` limite le vocabulaire à 200 mots les plus fréquents pour éviter le surapprentissage.
    # - `ngram_range=(1, 2)` permet de calculer les unigrammes (mots individuels) et les bigrammes (séquences de deux mots).
    ('vectorizer', TfidfVectorizer(max_features=200, ngram_range=(1, 2))),

    # Étape 2 : Classifieur k plus proches voisins (KNeighborsClassifier)
    # - `n_neighbors=3` indique qu'on utilise les 3 voisins les plus proches pour effectuer la classification.
    # - `weights='distance'` donne plus de poids aux voisins plus proches lors de la décision.
    ('classifier', KNeighborsClassifier(n_neighbors=3, weights='distance'))
    ])

# Entraînement du modèle
# `fit` entraîne le pipeline entier (vectorisation suivie de la classification) sur les données d'entraînement.
model_pipeline.fit(x, y_encoded)

# Génération des prédictions et évaluation du modèle sur les données de test
# - `predict` applique le traitement aux données de test et génère les prédictions sous forme binaire
predictions = model_pipeline.predict(x_test)

# Affichage des résultats
# 1) Calcul et affichage de la précision du modèle sur les données de test.
#    La fonction `accuracy_score` évalue la similarité entre les prédictions et les labels réels.
print(f"Test Accuracy: {accuracy_score(y_test_encoded, predictions):.2f}")

# 2) Affichage d'un rapport de classification détaillé.
#    La fonction `classification_report` fournit des métriques comme la précision, le rappel et le F1-score
#    pour chaque classe cible (interprétée à partir des classes encodées).
print("Classification Report:")
print(classification_report(y_test_encoded,
                            predictions,
                            zero_division=0,
                            target_names=encoder.classes_))